In [ ]:
pip install deltacat getdaft

In [ ]:
from daft import DataType, Schema, DataFrame, col 


multimodal_data_schema = Schema({
    "data": DataType.struct(
        DataType.string(), # text
        DataType.image(), # image
        DataType.string(), # audio
        DataType.string(), # video
        DataType.timestamp('ns'), # timestamp
    ),
})

metadata_schema = Schema({
    "id": DataType.string(),
    "name": DataType.string(),
    "description": DataType.string(),
    "created_at": DataType.timestamp('ns'),
    "updated_at": DataType.timestamp('ns'),
    "inserted_at": DataType.timestamp('ns'),
    "version": DataType.string(),
    "size_bytes": DataType.uint64(),
    "checksum": DataType.string(),
    "mime_type": DataType.string(),

})

embeddings_schema = Schema({
    "embeddings": DataType.embedding(),
    "embedding_metadata": metadata_schema,
})

base_schema = Schema({
    "data": 
    "metadata": metadata_schema,
    "embeddings": DataType.embedding(),
    "embedding_metadata": metadata_schema,

})

metadata_df = DataFrame(
    data=None,
    schema=metadata_schema,
    partition_by=["id"],
    mode="append",
    name="metadata",
)

DataType.struct(
        DataType.string(), # id
        DataType.string(), # name
        DataType.string(), # description
        DataType.timestamp('ns'), # created_at
        DataType.timestamp('ns'), # updated_at
        DataType.timestamp('ns'), # inserted_at
        DataType.string(), # version
        DataType.uint64(), # size_bytes
        DataType.string(), # checksum
        DataType.string(), # mime_type
    ),


In [ ]:
from daft import DataType, Schema, DataFrame, TimeUnit, col
import daft


image_schema = Schema({
    "": DataType.string(),
    "width": DataType.uint64(),
    "height": DataType.uint64(),
})

# Base Node Schema
base_node_schema = Schema({
    "content": DataType.struct(
        DataType.string(), # text
        DataType.image(), # image
        DataType.string(), # audio
        DataType.string(), # video
        DataType.timestamp('ns'), # timestamp
    ),
    "type": DataType.string(),
    "url": DataType.string(),
    "timestamp": DataType.timestamp('ns'),
    "metadata": DataType.struct(
        DataType.string(), # id
        DataType.string(), # name
        DataType.string(), # description
        DataType.timestamp('ns'), # created_at
        DataType.timestamp('ns'), # updated_at
        DataType.timestamp('ns'), # inserted_at
        DataType.string(), # version
    )
})

# Edge Schema
edge_schema = Schema({
    "source": DataType.string(),
    "target": DataType.string(),
    "relation": DataType.string(),
    "timestamp": DataType.timestamp('ns'),
    "metadata": DataType.struct(
        DataType.string(), # id
        DataType.string(), # name
        DataType.string(), # description
        DataType.timestamp('ns'), # created_at
        DataType.timestamp('ns'), # updated_at
        DataType.timestamp('ns'), # inserted_at
        DataType.string(), # version
    )
})

class BaseStructure:
    def __init__(self, index: str):
        self.index = index
        self.nodes = DataFrame(schema=base_node_schema)

    def add_node(self, nodes: DataFrame):
        new_node = DataFrame.from_pydict(node_data)
        self.nodes = self.nodes.concat(new_node)
        return self

    def get(self, ):
        return self.nodes.collect()

    def filter(self, condition):
        """
        Filter the nodes DataFrame.

        Args:
            condition (str): A condition to filter the nodes.
        """
        self.nodes = self.nodes.filter(condition)
        return self

    def upsert(self, nodes: DataFrame):
        """
        Upsert a node into the nodes DataFrame.

        Args:
            nodes (DataFrame): A DataFrame of nodes to upsert.
        """
        condition = self.nodes.select("metadata.id") == nodes.select("metadata.id")
        self.nodes = self.nodes.filter(nodes, condition)
        return self
                                                
    def sql(self, query: str):
        return self.nodes.sql(query)

class EdgeStructure:
    def __init__(self, index: str):
        self.index = index
        self.edges = DataFrame(schema=edge_schema)

    def add_edge(self, edge_data: dict):
        new_edge = DataFrame.from_pydict(edge_data)
        self.edges = self.edges.concat(new_edge)
        return self
    
    def get_edges(self):
        return self.edges

    def filter_edges(self, condition):
        self.edges = self.edges.filter(condition)
        return self

    def upsert_edges(self, source: str, target: str, update_dict: dict):
        condition = 

class Document:
    def __init__(self, index: str):
        self.base = BaseStructure(index)

    def add_text(self, id: str, content: str):
        return self.base.add_node({"id": id, "content": content, "type": "text"})

    def add_image(self, id: str, content: str, url: str):
        return self.base.add_node({"id": id, "content": content, "type": "image", "url": url})

    def add_audio(self, id: str, content: str, url: str, duration: float):
        return self.base.add_node({"id": id, "content": content, "type": "audio", "url": url, "duration": duration})

    def add_video(self, id: str, content: str, url: str, duration: float):
        return self.base.add_node({"id": id, "content": content, "type": "video", "url": url, "duration": duration})

    def add_timestamp(self, id: str, content: str, timestamp):
        return self.base.add_node({"id": id, "content": content, "type": "timestamp", "timestamp": timestamp})

    def get_nodes(self):
        return self.base.get_nodes()

    def filter_nodes(self, condition):
        self.base.filter_nodes(condition)
        return self

    def update_node(self, node_id: str, update_dict: dict):
        self.base.update_node(node_id, update_dict)
        return self

class Graph:
    def __init__(self, index: str):
        self.base = BaseStructure(index)
        self.edges = DataFrame(schema=edge_schema)

    def add_node(self, node_data: dict):
        self.base.add_node(node_data)
        return self

    def add_edge(self, edge_data: dict):
        new_edge = DataFrame.from_pydict(edge_data)
        self.edges = self.edges.concat(new_edge)
        return self

    def get_nodes(self):
        return self.base.get_nodes()

    def get_edges(self):
        return self.edges

    def filter_nodes(self, condition):
        self.base.filter_nodes(condition)
        return self

    def filter_edges(self, condition):
        self.edges = self.edges.filter(condition)
        return self

    def update_node(self, node_id: str, update_dict: dict):
        self.base.update_node(node_id, update_dict)
        return self

    def update_edge(self, source: str, target: str, update_dict: dict):
        condition = (self.edges["source"] == source) & (self.edges["target"] == target)
        for key, value in update_dict.items():
            self.edges = self.edges.with_column(key, 
                daft.where(condition, daft.lit(value), self.edges[key]))
        return self

In [ ]:
# Create and populate a document
doc = (Document("doc1")
    .add_text("1", "This is a document")
    .add_image("2", "An image in the document", "http://example.com/image.jpg")
    .add_audio("3", "An audio clip", "http://example.com/audio.mp3", 120.5)
    .filter_nodes(col("type") != "audio"))

# Create and populate a graph
graph = (Graph("graph1")
    .add_node({"id": "1", "content": "Hello, world!", "type": "text"})
    .add_node({"id": "2", "content": "A beautiful landscape", "type": "image", "url": "http://example.com/image.jpg"})
    .add_edge({"source": "1", "target": "2", "relation": "describes"})
    .update_node("1", {"content": "Updated content"})
    .filter_edges(col("relation") == "describes"))

# Get nodes and edges
doc_nodes = doc.get_nodes()
graph_nodes = graph.get_nodes()
graph_edges = graph.get_edges()

In [ ]:
# Hierarchical Task Network,:
class HTN():
    def __init__(self, index):
        self.graph = Graph(index)

    def add_node(self, node_data: dict):
        new_node = DataFrame.from_pydict(node_data)
        self.nodes = self.nodes.concat(new_node)
        return self

    def add_edge(self, edge_data: dict):
        new_edge = DataFrame.from_pydict(edge_data)
        self.edges = self.edges.concat(new_edge)
        return self

    def get_nodes(self):
        return self.nodes
